In [1]:
!python3 -m pip install --upgrade pip

In [2]:
!pip install torchvision timm

In [3]:
#import library
import torch
import timm

# Define the function for loading the model
def load_efficientnet_b0(model_path, num_classes=39, device=None):
    """
    load model
    
    parameter:
        model_path: 
        num_classes: 39
        device: 
    
    return:
        model: The loaded model object
    """
    # detect device
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # create same structure
    model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=num_classes)
    
    # load weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # Move the model to the specified device and set it to evaluation mode
    model = model.to(device)
    model.eval()
    
    return model

# use case

model_path = "EfficientNet-B0.pth"
    
    # load model
model = load_efficientnet_b0(model_path)
print("The model has been successfully loaded")#the model has been successfully loaded

The model has been successfully loaded


In [4]:
# 植物病害类别列表（39类，最后一类是背景）
# Plant disease class list (39 classes, last is 'background')
disease_names = [
    'Apple___Apple_scab',  # 苹果黑星病 Apple scab
    'Apple___Black_rot',  # 苹果黑腐病 Apple Black rot
    'Apple___Cedar_apple_rust',  # 苹果雪松锈病 Apple Cedar apple rust
    'Apple___healthy',  # 苹果健康 Apple healthy
    'Blueberry___healthy',  # 蓝莓健康 Blueberry healthy
    'Cherry_(including_sour)___Powdery_mildew',  # 樱桃白粉病 Cherry Powdery mildew
    'Cherry_(including_sour)___healthy',  # 樱桃健康 Cherry healthy
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  # 玉米灰斑病 Corn Gray leaf spot
    'Corn_(maize)___Common_rust_',  # 玉米普通锈病 Corn Common rust
    'Corn_(maize)___Northern_Leaf_Blight',  # 玉米北方叶斑病 Corn Northern Leaf Blight
    'Corn_(maize)___healthy',  # 玉米健康 Corn healthy
    'Grape___Black_rot',  # 葡萄黑腐病 Grape Black rot
    'Grape___Esca_(Black_Measles)',  # 葡萄腐烂病 Grape Esca (Black Measles)
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  # 葡萄叶枯病 Grape Leaf blight (Isariopsis Leaf Spot)
    'Grape___healthy',  # 葡萄健康 Grape healthy
    'Orange___Haunglongbing_(Citrus_greening)',  # 橙黄龙病 Orange Huanglongbing (Citrus greening)
    'Peach___Bacterial_spot',  # 桃细菌性斑点 Peach Bacterial spot
    'Peach___healthy',  # 桃健康 Peach healthy
    'Pepper,_bell___Bacterial_spot',  # 灯笼椒细菌性斑点 Pepper, bell Bacterial spot
    'Pepper,_bell___healthy',  # 灯笼椒健康 Pepper, bell healthy
    'Potato___Early_blight',  # 马铃薯早疫病 Potato Early blight
    'Potato___Late_blight',  # 马铃薯晚疫病 Potato Late blight
    'Potato___healthy',  # 马铃薯健康 Potato healthy
    'Raspberry___healthy',  # 树莓健康 Raspberry healthy
    'Soybean___healthy',  # 大豆健康 Soybean healthy
    'Squash___Powdery_mildew',  # 南瓜白粉病 Squash Powdery mildew
    'Strawberry___Leaf_scorch',  # 草莓叶灼病 Strawberry Leaf scorch
    'Strawberry___healthy',  # 草莓健康 Strawberry healthy
    'Tomato___Bacterial_spot',  # 番茄细菌性斑点 Tomato Bacterial spot
    'Tomato___Early_blight',  # 番茄早疫病 Tomato Early blight
    'Tomato___Late_blight',  # 番茄晚疫病 Tomato Late blight
    'Tomato___Leaf_Mold',  # 番茄叶霉病 Tomato Leaf Mold
    'Tomato___Septoria_leaf_spot',  # 番茄叶斑病 Tomato Septoria leaf spot
    'Tomato___Spider_mites Two-spotted_spider_mite',  # 番茄二斑叶螨 Tomato Spider mites (Two-spotted spider mite)
    'Tomato___Target_Spot',  # 番茄靶斑病 Tomato Target Spot
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',  # 番茄黄化卷叶病毒病 Tomato Yellow Leaf Curl Virus
    'Tomato___Tomato_mosaic_virus',  # 番茄花叶病毒病 Tomato mosaic virus
    'Tomato___healthy',  # 番茄健康 Tomato healthy
    'background'  # 背景背景（非植物）Background (non-plant)
]


In [15]:
!ls -l /dev/video*

crw-rw---- 1 root video 81, 0 May  1 00:54 /dev/video0


In [9]:
# —— 1. Necessary Imports ——
import cv2
import io
import time
import torch
from PIL import Image
import IPython
from torchvision import transforms

# —— 2. Global Constants ——
FONT           = cv2.FONT_HERSHEY_SIMPLEX  # Font for text
FONT_SCALE     = 0.7                        # Scale factor for text
COLOR          = (0, 255, 0)                # Text color (BGR): Green
THICKNESS      = 2                          # Line thickness for text
PRED_INTERVAL  = 0.5                        # Minimum interval between predictions (seconds)

# —— 3. Display Utility ——
def show_array(a, fmt='jpeg'):
    """
    Display an RGB image array inside Jupyter Notebook.
    """
    buf = io.BytesIO()
    Image.fromarray(a).save(buf, fmt)
    display(IPython.display.Image(data=buf.getvalue()))

# —— 4. Real-Time Detection Function ——
def live_plant_disease_detection(model, device, disease_names, cam_id=0, pred_interval=PRED_INTERVAL):
    """
    Perform real-time Top-3 classification using a preloaded model and webcam.

    Parameters:
      model         (torch.nn.Module): Model that has been loaded with state_dict and set to eval()
      device        (torch.device)   : Device where the model resides, e.g., torch.device('cuda:0')
      disease_names (List[str])      : Mapping from index to class name (aligned with ImageFolder.classes)
      cam_id        (int)            : Camera ID (default is 0)
      pred_interval (float)          : Time interval between predictions, in seconds
    """
    # 1) Preprocessing pipeline: BGR → RGB → PIL → Resize → Tensor → Normalize
    preprocess = transforms.Compose([
        transforms.Lambda(lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2RGB)),
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        ),
    ])

    # 2) Open camera
    cap = cv2.VideoCapture(cam_id)
    if not cap.isOpened():
        print("Failed to open camera")
        return

    print("Starting real-time detection. Press Ctrl+C to stop.")
    last_t = time.time()
    preds  = []  # Stores current Top-3 predictions

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Frame read failed, exiting")
                break

            now = time.time()
            # Run prediction at intervals
            if now - last_t >= pred_interval:
                inp = preprocess(frame).unsqueeze(0).to(device)
                with torch.no_grad():
                    logits = model(inp)
                    probs  = torch.softmax(logits, dim=1)[0]
                    top_p, top_i = torch.topk(probs, 3)
                    preds = [
                        (idx.item(), disease_names[idx.item()], top_p[j].item())
                        for j, idx in enumerate(top_i)
                    ]
                last_t = now

            # Visualization
            disp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            for i, (_, name, p) in enumerate(preds):
                plant, cond = name.split('___') if '___' in name else (name, '')
                plant = plant.replace('_',' ')
                cond  = cond .replace('_',' ')
                text = f"{plant}: {cond} ({p:.2f})"
                cv2.putText(disp, text, (10, 30 + i*30),
                            FONT, FONT_SCALE, COLOR, THICKNESS)

            show_array(disp)
            IPython.display.clear_output(wait=True)

    except KeyboardInterrupt:
        print("Real-time detection stopped.")
    finally:
        cap.release()

# Run this function directly:
live_plant_disease_detection(
    model=model,
    device=device,
    disease_names=disease_names,
    cam_id=0,            # Change to your camera ID
    pred_interval=0.5    # Predict once every 0.5 seconds
)


Real-time detection stopped.


In [ ]:
# 39 plant disease classes with prevention & treatment suggestions (English only)
treatment_dict = {
    "Apple___Apple_scab": "Remove diseased leaves/fruits, improve ventilation, and spray appropriate fungicides.",
    "Apple___Black_rot": "Prune infected branches and apply fungicides during autumn/winter cleanup.",
    "Apple___Cedar_apple_rust": "Plant resistant varieties, prune regularly, and prevent cross-infection.",
    "Apple___healthy": "Plant is healthy, no treatment needed.",
    "Blueberry___healthy": "Plant is healthy, no treatment needed.",
    "Cherry_(including_sour)___Powdery_mildew": "Increase ventilation, remove infected leaves, and spray fungicides like myclobutanil.",
    "Cherry_(including_sour)___healthy": "Plant is healthy, no treatment needed.",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": "Use resistant varieties, remove infected residues, and practice crop rotation.",
    "Corn_(maize)___Common_rust_": "Remove diseased plants promptly and apply fungicides like triadimefon.",
    "Corn_(maize)___Northern_Leaf_Blight": "Use resistant varieties, enhance field management, and apply fungicides in time.",
    "Corn_(maize)___healthy": "Plant is healthy, no treatment needed.",
    "Grape___Black_rot": "Prune diseased branches, clean fallen leaves/fruits, and spray Bordeaux mixture.",
    "Grape___Esca_(Black_Measles)": "Prune and burn infected branches, maintain vineyard hygiene.",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": "Remove infected leaves, prune properly, and apply fungicides.",
    "Grape___healthy": "Plant is healthy, no treatment needed.",
    "Orange___Haunglongbing_(Citrus_greening)": "Remove infected trees, control psyllid vectors, and use healthy seedlings.",
    "Peach___Bacterial_spot": "Avoid wounds, plant resistant varieties, and spray copper-based fungicides.",
    "Peach___healthy": "Plant is healthy, no treatment needed.",
    "Pepper,_bell___Bacterial_spot": "Remove diseased plants, improve management, and apply copper fungicides.",
    "Pepper,_bell___healthy": "Plant is healthy, no treatment needed.",
    "Potato___Early_blight": "Practice crop rotation, plant properly, and apply chlorothalonil fungicide.",
    "Potato___Late_blight": "Use resistant varieties, apply fungicides promptly, and avoid field water accumulation.",
    "Potato___healthy": "Plant is healthy, no treatment needed.",
    "Raspberry___healthy": "Plant is healthy, no treatment needed.",
    "Soybean___healthy": "Plant is healthy, no treatment needed.",
    "Squash___Powdery_mildew": "Improve ventilation, control humidity, and spray sulfur-based fungicides.",
    "Strawberry___Leaf_scorch": "Remove infected leaves, avoid excessive moisture, and use fungicides when needed.",
    "Strawberry___healthy": "Plant is healthy, no treatment needed.",
    "Tomato___Bacterial_spot": "Use healthy seedlings, remove diseased leaves, and spray copper fungicides.",
    "Tomato___Early_blight": "Practice crop rotation, spray fungicides promptly, and improve ventilation.",
    "Tomato___Late_blight": "Plant resistant varieties, avoid high humidity, and apply fungicides promptly.",
    "Tomato___Leaf_Mold": "Control humidity, remove diseased leaves, and use special fungicides.",
    "Tomato___Septoria_leaf_spot": "Proper spacing, reduce water splash, and spray fungicides timely.",
    "Tomato___Spider_mites Two-spotted_spider_mite": "Apply acaricides and keep the field clean.",
    "Tomato___Target_Spot": "Improve ventilation and apply fungicides promptly.",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": "Remove diseased plants, control aphids, and plant resistant varieties.",
    "Tomato___Tomato_mosaic_virus": "Avoid mechanical injuries and use healthy seedlings.",
    "Tomato___healthy": "Plant is healthy, no treatment needed.",
    "background": "No plant or disease detected."
}
